<a href="https://colab.research.google.com/github/yashjangra07/WordForThought/blob/main/Week%203/Yash/Audio_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle
! mkdir /content/Kaggle_API
! mkdir /content/Kaggle_API/.kaggle
from google.colab import files
files.upload()
! ls /content/Kaggle_API -a
! mv /content/kaggle.json /content/Kaggle_API/.kaggle
! ls /content/Kaggle_API/.kaggle -a
! chmod 600 /content/Kaggle_API/.kaggle/kaggle.json
! kaggle datasets download uwrfkaggler/ravdess-emotional-speech-audio
! kaggle datasets download ejlok1/cremad

Saving kaggle.json to kaggle.json
.  ..  .kaggle
.  ..  kaggle.json
Dataset URL: https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio
License(s): CC-BY-NC-SA-4.0
100% 429M/429M [00:20<00:00, 22.8MB/s]
100% 429M/429M [00:20<00:00, 22.2MB/s]
Dataset URL: https://www.kaggle.com/datasets/ejlok1/cremad
License(s): ODC Attribution License (ODC-By)
100% 451M/451M [00:20<00:00, 24.5MB/s]
100% 451M/451M [00:20<00:00, 22.9MB/s]


In [2]:
! unzip ravdess-emotional-speech-audio.zip

Archive:  ravdess-emotional-speech-audio.zip
  inflating: Actor_01/03-01-01-01-01-01-01.wav  
  inflating: Actor_01/03-01-01-01-01-02-01.wav  
  inflating: Actor_01/03-01-01-01-02-01-01.wav  
  inflating: Actor_01/03-01-01-01-02-02-01.wav  
  inflating: Actor_01/03-01-02-01-01-01-01.wav  
  inflating: Actor_01/03-01-02-01-01-02-01.wav  
  inflating: Actor_01/03-01-02-01-02-01-01.wav  
  inflating: Actor_01/03-01-02-01-02-02-01.wav  
  inflating: Actor_01/03-01-02-02-01-01-01.wav  
  inflating: Actor_01/03-01-02-02-01-02-01.wav  
  inflating: Actor_01/03-01-02-02-02-01-01.wav  
  inflating: Actor_01/03-01-02-02-02-02-01.wav  
  inflating: Actor_01/03-01-03-01-01-01-01.wav  
  inflating: Actor_01/03-01-03-01-01-02-01.wav  
  inflating: Actor_01/03-01-03-01-02-01-01.wav  
  inflating: Actor_01/03-01-03-01-02-02-01.wav  
  inflating: Actor_01/03-01-03-02-01-01-01.wav  
  inflating: Actor_01/03-01-03-02-01-02-01.wav  
  inflating: Actor_01/03-01-03-02-02-01-01.wav  
  inflating: Actor_01/03

In [3]:
! unzip cremad.zip

Streaming output truncated to the last 5000 lines.
  inflating: AudioWAV/1031_DFA_DIS_XX.wav  
  inflating: AudioWAV/1031_DFA_FEA_XX.wav  
  inflating: AudioWAV/1031_DFA_HAP_XX.wav  
  inflating: AudioWAV/1031_DFA_NEU_XX.wav  
  inflating: AudioWAV/1031_DFA_SAD_XX.wav  
  inflating: AudioWAV/1031_IEO_ANG_HI.wav  
  inflating: AudioWAV/1031_IEO_ANG_LO.wav  
  inflating: AudioWAV/1031_IEO_ANG_MD.wav  
  inflating: AudioWAV/1031_IEO_DIS_HI.wav  
  inflating: AudioWAV/1031_IEO_DIS_LO.wav  
  inflating: AudioWAV/1031_IEO_DIS_MD.wav  
  inflating: AudioWAV/1031_IEO_FEA_HI.wav  
  inflating: AudioWAV/1031_IEO_FEA_LO.wav  
  inflating: AudioWAV/1031_IEO_FEA_MD.wav  
  inflating: AudioWAV/1031_IEO_HAP_HI.wav  
  inflating: AudioWAV/1031_IEO_HAP_LO.wav  
  inflating: AudioWAV/1031_IEO_HAP_MD.wav  
  inflating: AudioWAV/1031_IEO_NEU_XX.wav  
  inflating: AudioWAV/1031_IEO_SAD_HI.wav  
  inflating: AudioWAV/1031_IEO_SAD_LO.wav  
  inflating: AudioWAV/1031_IEO_SAD_MD.wav  
  inflating: AudioWAV/103

In [7]:
! rm -rf Actor_01 Actor_02 Actor_03 Actor_04 Actor_05 Actor_06 Actor_07 Actor_08 Actor_09 Actor_10 Actor_11 Actor_12 Actor_13 Actor_14
! rm -rf Actor_15 Actor_16 Actor_17 Actor_18 Actor_19 Actor_20 Actor_21 Actor_22 Actor_23 Actor_24 8

In [8]:
import pandas as pd
import numpy as np
import os
import librosa
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split


Ravdess = "/content/audio_speech_actors_01-24/"
ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []
file_path = []
for dir in ravdess_directory_list:

    if os.path.isdir(os.path.join(Ravdess, dir)):
        actor = os.listdir(Ravdess + dir)
        for file in actor:
            part = file.split('.')[0].split('-')
            file_emotion.append(int(part[2]))
            file_path.append(Ravdess + dir + '/' + file)

emotion_df = pd.DataFrame(file_emotion, columns=['Emotion'])
path_df = pd.DataFrame(file_path, columns=['Path'])
ravdess_df = pd.concat([emotion_df, path_df], axis=1)

ravdess_df.Emotion.replace({1: 'Neutral', 2: 'Calm', 3: 'Happy', 4: 'Sad', 5: 'Angry', 6: 'Fear', 7: 'Disgust', 8: 'Surprise'}, inplace=True)

Crema = "/content/AudioWAV/"
crema_directoryList = os.listdir(Crema)

file_emotion = []
file_path = []

for file in crema_directoryList:
    file_path.append(Crema + file)
    part = file.split('_')

    if part[2] == 'ANG':
        file_emotion.append('Angry')
    elif part[2] == 'DIS':
        file_emotion.append("Disgust")
    elif part[2] == 'FEA':
        file_emotion.append("Fear")
    elif part[2] == 'HAP':
        file_emotion.append('Happy')
    elif part[2] == 'NEU':
        file_emotion.append('Neutral')
    else:
        file_emotion.append('Unknown')

emotion_df = pd.DataFrame(data=file_emotion, columns=['Emotion'])
path_df = pd.DataFrame(data=file_path, columns=['Path'])
crema_df = pd.concat([emotion_df, path_df], axis=1)

final_df = pd.concat([ravdess_df, crema_df], axis=0)
final_df.to_csv("final_df.csv", index=False)

<ipython-input-8-7a9880317a0b>:27: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ravdess_df.Emotion.replace({1: 'Neutral', 2: 'Calm', 3: 'Happy', 4: 'Sad', 5: 'Angry', 6: 'Fear', 7: 'Disgust', 8: 'Surprise'}, inplace=True)


In [9]:
def extract_features(data, sampling_rate):
    result = np.array([])
    MFCC = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate).T, axis=0)
    result = np.hstack((result, MFCC))
    mel_spec = np.mean(librosa.feature.melspectrogram(y=data, sr=sampling_rate).T, axis=0)
    result = np.hstack((result, mel_spec))
    return result


X, Y = [], []
for path, emotion in zip(final_df.Path, final_df.Emotion):
    data, sampling_rate = librosa.load(path, duration=2.5, offset=0.6)
    feature = extract_features(data, sampling_rate)
    X.append(feature)
    Y.append(emotion)


encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1, 1)).toarray()


X = np.array(X)
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0, shuffle=True)


x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

In [10]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout

from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout

model = Sequential()

model.add(Conv1D(256, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(x_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
model.add(Conv1D(128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
model.add(Conv1D(64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(9, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()


history = model.fit(x_train, y_train, batch_size=16, epochs=40, validation_data=(x_test, y_test))

print("Accuracy of the model on the test data:", model.evaluate(x_test, y_test)[1] * 100, '%')


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 148, 256)            │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 74, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 74, 128)             │          98,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 37, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 37, 64)              │          24,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 19, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          77,888 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 9)                   │             297 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 204,361 (798.29 KB)

 Trainable params: 204,361 (798.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 14s 13ms/step - accuracy: 0.2066 - loss: 2.2229 - val_accuracy: 0.3476 - val_loss: 1.7276
Epoch 2/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.2621 - loss: 1.7985 - val_accuracy: 0.3593 - val_loss: 1.6213
Epoch 3/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.2840 - loss: 1.7124 - val_accuracy: 0.3561 - val_loss: 1.5993
Epoch 4/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.3020 - loss: 1.6633 - val_accuracy: 0.3503 - val_loss: 1.5628
Epoch 5/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.3164 - loss: 1.6503 - val_accuracy: 0.3624 - val_loss: 1.5519
Epoch 6/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.3251 - loss: 1.6374 - val_accuracy: 0.3787 - val_loss: 1.5198
Epoch 7/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.3386 - loss: 1.6150 - val_accuracy: 0.3652 - val_loss: 1.5515
Epoch 8/40
417/417 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.3542 - loss: 1.5693 - val_accuracy:

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout

from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout

model = Sequential()

model.add(Conv1D(256, kernel_size=3, strides=1, padding='same', activation='relu', input_shape=(x_test.shape[1], 1)))
model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
model.add(Conv1D(128, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
model.add(Conv1D(64, kernel_size=3, strides=1, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3, strides=2, padding='same'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(9, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()


history = model.fit(x_test, y_test, batch_size=16, epochs=40, validation_data=(x_train, y_train))

print("Accuracy of the model on the test data:", model.evaluate(x_train, y_train)[1] * 100, '%')


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)                    │ (None, 148, 256)            │           1,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 74, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 74, 128)             │          98,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_4 (MaxPooling1D)       │ (None, 37, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 37, 64)              │          24,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_5 (MaxPooling1D)       │ (None, 19, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 1216)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │          77,888 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 9)                   │             297 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 204,361 (798.29 KB)

 Trainable params: 204,361 (798.29 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/40
139/139 ━━━━━━━━━━━━━━━━━━━━ 12s 54ms/step - accuracy: 0.1643 - loss: 2.5241 - val_accuracy: 0.2220 - val_loss: 1.9407
Epoch 2/40
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2121 - loss: 1.9871 - val_accuracy: 0.2964 - val_loss: 1.8356
Epoch 3/40
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2160 - loss: 1.9040 - val_accuracy: 0.3322 - val_loss: 1.7200
Epoch 4/40
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2473 - loss: 1.8093 - val_accuracy: 0.3253 - val_loss: 1.6762
Epoch 5/40
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.2826 - loss: 1.7355 - val_accuracy: 0.3253 - val_loss: 1.6806
Epoch 6/40
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3032 - loss: 1.7079 - val_accuracy: 0.3238 - val_loss: 1.6218
Epoch 7/40
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.3170 - loss: 1.6422 - val_accuracy: 0.3423 - val_loss: 1.5991
Epoch 8/40
139/139 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.2936 - loss: 1.6787 - val_accuracy: 